Hunting for iso in the fgdc. Specifically CI_OnlineResources in the idinfo citations.

I'm going to store the output in an appropriate way - with some XML in the JSON.

In [1]:
%reload_ext autoreload
%autoreload 2

import json as js  # name conflict with sqla
import sqlalchemy as sqla
from sqlalchemy.orm import sessionmaker

from mpp.models import Response

from lxml import etree

In [2]:
def _extract_tag(t):
    if not t:
        return
    return t.split('}')[-1]

def _taggify(e):
    tags = [e.tag] + [m.tag for m in e.iterancestors()]
    tags.reverse()

    try:
        return [_extract_tag(t) for t in tags]
    except:
        return []

def extract_resources(xml):
    elems = xml.xpath('//*/CI_OnlineResource')
    
    for elem in elems:
        tag = '/'.join(_taggify(elem))
        resource = etree.tostring(elem)
        yield tag, resource

In [3]:
# grab the clean text from the rds
with open('../local/big_rds.conf', 'r') as f:
    conf = js.loads(f.read())

# our connection
engine = sqla.create_engine(conf.get('connection'))
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

In [4]:
sketchy_sql = '''with i
as (
    select d.response_id, jsonb_array_elements(d.identity::jsonb) ident
    from identities d
    where d.identity is not null
)

select r.id, r.cleaned_content
from responses r join i on i.response_id = r.id
where i.ident->>'protocol' = 'FGDC'
limit %s
offset %s;
'''

In [21]:
session.rollback()

In [ ]:
with open('outputs/fgdc_with_iso.txt', 'w') as f:
    f.write('[\n')

# s, e, i = 0, 10, 10
s, e, i = 0, 26299, 100

for x in xrange(s, e, i):
    for response in session.execute(sketchy_sql % (i, x)):
        
        try:
            xml = etree.fromstring(response['cleaned_content'].encode('utf-8'))
        except:
            print 'failed xml', response['id']
            continue
        
        resources = [(t, c) for t, c in extract_resources(xml)]
        
        if resources:
            with open('outputs/fgdc_with_iso.txt', 'a') as f:
                f.write(js.dumps([{"id": response['id'], "tags": t, "xml": r} for t, r in resources]) + ',\n')

                
with open('outputs/fgdc_with_iso.txt', 'a') as f:
    f.write(']')